In [3]:
import json
import numpy as np
import os

### 센서 데이터의 fall_start_frame과 fall_end_frame 정보를 기존 json 파일에 가져오기

In [4]:
def addition_sensordata_json(sensor_folder, main_folder, output_folder) :
    main_files = [f for f in os.listdir(main_folder) if f.endswith('.json')]
    
    for file in main_files : 
        main_file_path = os.path.join(main_folder, file)
        sensor_file_path = os.path.join(sensor_folder, file)
        output_file_path = os.path.join(output_folder, file)
        
        if os.path.exists(sensor_file_path) and os.path.exists(main_file_path) : 
            with open(sensor_file_path, 'r', encoding = 'utf-8') as f : 
                sensor_data = json.load(f)
                Sensordata = sensor_data.get('sensordata', {})
                
            with open(main_file_path, 'r', encoding = 'utf-8') as f : 
                main_data = json.load(f)
            
            main_data['sensordata'] = Sensordata 
            
            with open(output_file_path, 'w', encoding = 'utf-8') as f : 
                json.dump(main_data, f, ensure_ascii = False, indent = 2)
            
            print('수정 및 저장중')
        else : 
            print('파일이 없어서 건너뜀')
            
sensor_folder = 'D:\\human_fall\\re_landmark\\sensor_json'
main_folder = 'D:\\human_fall\\re_landmark\\re_train_NY_json'
output_folder = 'D:\\human_fall\\re_landmark\\addition_json'

addition_sensordata_json(sensor_folder, main_folder, output_folder)

수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수정 및 저장중
수

### 가져온 fall_start_frame과 fall_end_frame을 기준으로 클래스 재분류

In [ ]:
# 클래스 수정 함수
def re_classify_json(main_file, modified_file) : 
    with open(main_file, 'r', encoding = 'utf-8') as f : 
        data = json.load(f)
    
    fall_start_frame = data.get('sensordata', {}).get('fall_start_frame', -1)
    fall_end_frame = data.get('sensordata', {}).get('fall_end_frame', -1)
    pose_data = data['pose_data']
    
    # fall_start_frame과 fall_end_frame을 기준으로 
    # fall_start_frame = 0 or fall_end_frame = 0 이면 Normal, 
    # 또한 fall_start_frame 이하는 Normal,
    # fall_start_frame과 fall_end_frame까지는 Danger,
    # fall_end_frame 이후는 Fall 클래스로 
    if fall_start_frame == 0 or fall_end_frame == 0:
        for frame_key, frame_data in pose_data.items():
            frame_data['class'] = 'Normal'
    else:
        for frame_key, frame_data in pose_data.items():
            frame = int(frame_key.split('_')[1])
            if frame < fall_start_frame:
                new_class = 'Normal'
            elif fall_start_frame <= frame <= fall_end_frame:
                new_class = 'Danger'
            else:
                new_class = 'Fall'
            
            frame_data['class'] = new_class
        
    with open(modified_file, 'w', encoding = 'utf-8') as f : 
        json.dump(data, f, ensure_ascii = False, indent = 2)     

# 저장
def addition_classes(input_folder, output_folder) : 
    for filename in os.listdir(input_folder) : 
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        re_classify_json(input_path, output_path)

input_folder = 'D:\\human_fall\\re_landmark\\addition_json'
output_folder = 'D:\\human_fall\\re_landmark\\addition_json'
addition_classes(input_folder, output_folder)

In [ ]:
# 연속된 프레임간의 평균 변화량 계산 
def calculate_movement(sequence):   
    diff = np.diff(sequence, axis=0)
    return np.mean(np.abs(diff))

def process_json_file(input_file, output_file, window_size=30, low_threshold=0.01, high_threshold=0.05):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    pose_data = data['pose_data']
    fall_start_frame = data.get('fall_start_frame', -1)
    fall_end_frame = data.get('fall_end_frame', -1)
    
    new_pose_data = {}
    landmark_history = []
    
    for frame, landmarks in pose_data.items():
        frame = int(frame)
        frame_landmarks = []
        for landmark in [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]:
            frame_landmarks.extend([
                landmarks[f'landmark_{landmark}']['x'],
                landmarks[f'landmark_{landmark}']['y']
            ])
        
        landmark_history.append(frame_landmarks)
        
        if len(landmark_history) > window_size:
            landmark_history.pop(0)
        
        if len(landmark_history) == window_size:
            movement = calculate_movement(np.array(landmark_history))
            
            # 클래스 분류
            if fall_start_frame <= frame <= fall_end_frame:
                class_label = "Fall Risk"
            elif movement > high_threshold:
                class_label = "Complete Fall"
            elif movement > low_threshold:
                class_label = "Fall Risk"
            else:
                class_label = "Non-Fall"
        else:
            movement = 0
            class_label = "Non-Fall"  # 초기 프레임들
        
        new_pose_data[frame] = {
            "landmarks": landmarks,
            "class": class_label,
            "movement": float(movement)
        }
    
    # 새로운 JSON 파일 생성
    new_data = {
        "pose_data": new_pose_data,
        "fall_start_frame": fall_start_frame,
        "fall_end_frame": fall_end_frame
    }
    
    with open(output_file, 'w') as f:
        json.dump(new_data, f, indent=4)

input_file = 'path/to/input.json'
output_file = 'path/to/output.json'
process_json_file(input_file, output_file)